In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests
import urllib.request as request
from urllib.parse import quote

import ruptures as rpt

from datetime import datetime
from datetime import timedelta
from tqdm import tqdm
import pyodbc 

pd.options.mode.chained_assignment = None

In [64]:
def get_token():
    refresh_token = 'eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ0b2tlblR5cGUiOiJUSElSRFBBUlRZQVBJX1JlZnJlc2giLCJ0b2tlbmlkIjoiMTU3ZDY4N2QtZDBjZS00OTcyLWI5MjQtMzJhMzY0ODFkYjYxIiwid2ViQXBwIjoiVGhpcmRQYXJ0eUFwcCIsInZlcnNpb24iOiIyIiwiaWRlbnRpdHlUb2tlbiI6Ik1sdDYzOTg2VS8vaUZ4aFhPSCthczVFbVh0U1FPUVBDTXorY1VqejdFSzNPTWx5VGVUbUdSUm9WZDZYa0RsMkR3S1NnSlVxTWZaejVHVjRYZ1F0Mzl5WnNoWEQvZUg3MjFkRzlzQzJDV2dSZC9STFNZZjNvSmhTRXdnRTdJbFZkMkRaSVY0SDlPTUpwOGh1VWtpK1VweFZIUDNwL09TTkhKVFJIekgyOE9CYlhwZmJQSEYveFd1NGhkK1p3RWRPSU8zZUFSK2EvVDFUcDc3cHFjOTU3aWxlb3plWDJmKzVtUk8xMnJaUTBIUG4wZFNTMzVlSEx0VVRwRG0vNmVCSDRNZUY5MENDaXMrc1ZoZDRJSE85ZWVyMkRPbGF1U2I5SVFkQk5XdDBQNEJoS3dOcWg0Y3lFMGZ4bWUvaDNXS1NYRVRDL0treWFaYSszNC9YVS94cWJ6a0J2NWhXSjhkYVZ2YTJlTE1jSHZpUkszRXRoMHVwMmxUT0lKZEpFWkJrL1JvcWhQelR2anVUNTZZZGEwZlp5VG5uYno5NkR0eVlqZC9tNlpuNDNzdDRJUmZFbVpzOWg0STU2b0ZoSkR4d0hFdnhra2IrMHhpYW1UMllhVjJyMTRKeDJ0dE1QR2N5S1RscllkVWE2a2pHYXlzRHI4aHhVdnlIYnBUeXFvd3V0bmZpYTQ2TnZOOVo3cUt1czhkK3E5YkgrV280RkQ2Ym1OMzMramRUWUFUY21TTWJya0kzWm9GSmxhYUFpWEJlMkdzRTNiVnFkcmxoTGtGTUtINFp5MGNuMTNzK21BTkd5eTZDcUdlQ2YweXVrYjJYUGpBb1oyNEdNN2dESjVKVmZjc2dPWDVMcURmMUZsUVdRbzRvd1pzSXBUd3ZMTWZXRXI5UnlETUJ3SkRZMnUwMWw2SmxQbllReU5ZRDd2ZkRtQjYwMVNaUm9CazBrMmxJMUp4SEk4UFRWVjNYT2Q1NWFaZDVXajJRMkdZcEd0NmlmNDNRZkV5aTMvd1k1QmpSMk1qNzg3NUhUUi9Xc2pqQlY5U0QwQmZ1ODVwT0J5WHVCenBRRmlCWHpzL0Z3cm9McWE4RHYyaTZxZEZ1UGZOM2QvSVJ3bmErRTlyQUdJRXZNZHV0dmREZE9nTEhsYm04NlE5aUU0NkhGcWdyd2JXY0tzMkdaK3dDSnhKclZaT2ZMSVQvQVVQcGNaKzdGbDdFNDNPT2dMSzdycEhFK2VJRitGRWZGeDAwMGVMekpERWdaTGdGRVVWMFFsRGtVZjR3bGtxNW1RaHNMNG5ualphb200dStzYVBraW5sV2NCTU5QdVFETzFZVFlhZTFZQmFKNkcyelo3OWU4aDhvQmQzM1RtMXNDSUorbTcwNXNjNm9nNUoyaEd1LzRtaXFEeE9pbjQ2ek5tS0djLzFqejlwT1I4MktrMXgxOXhWYm4vOXpKU1hnUGZSbEloZ2hQTVQrb3dEUXpaT2xJdWx4SVAxSGpib2I3TmhZRFl6T1ArMkdJU3JxS2k4dFBadXpYRnFZQ3RZN3RaMEFYTXRNQWVHbnhFNi9JNW9iKzI0VDVzRUN0S3c4T2RTb3ZJQlNCeU45R3YyczY0ZTNOWHBMNWZGcG80VGtoanZxYnhMNVhqR3FsZXc0cktjbDZmdFdRTTlGaTBXTEFERVZJek1McHpZRWhLSEtIQytHZDhidlI5UmI3SkNTZ0tySUVwZ1FyV2ZpVWcxazdQb3ZENVZCR04yT2l6dTR1Y1dkUmh1VDBkQTRQcXEyV0NTSCtYcVNzZHNnaWgxKzQ5c3pkTUJVck1BOGdvNENraDRjdVRNbmY4NTdWamVBWTV2aGp6NjM0ay9mOGxzZ2pxdGs3WHlOUUJjSk0ycU0vRjNrODY5OEt5V2l1NjM4RHhkMS9wV1lYdWEzQ0VtdlV2SlNxVUFpOGVHV1B1VngzcG9qeEFRNTVXZTgwSFUrbk93U01tYUJEZ0dKbGxsaFh6alZkNUd0TTNhcnYrNWRad3FaQWxVdWtwVVZYKy92ekg1MFNPU1M2c2Y4OTdwclNheHprQnpzdlFxWFpmTDFRc2I3dERMZDloa1R2cHZRbXl6d1ZJVlU9IiwiaHR0cDovL3NjaGVtYXMueG1sc29hcC5vcmcvd3MvMjAwNS8wNS9pZGVudGl0eS9jbGFpbXMvbmFtZWlkZW50aWZpZXIiOiJDVlI6MzMwNzc4MzEtUklEOjY0OTI5MzMxIiwiaHR0cDovL3NjaGVtYXMueG1sc29hcC5vcmcvd3MvMjAwNS8wNS9pZGVudGl0eS9jbGFpbXMvZ2l2ZW5uYW1lIjoiTWlra2VsIFJhc211cyBTY2htaWR0IiwibG9naW5UeXBlIjoiQ2VydGlmaWNhdGUiLCJyaWQiOiI2NDkyOTMzMSIsImN2ciI6IjMzMDc3ODMxIiwiY29tcGFueSI6Ik5SR2kgUsOlZGdpdm5pbmcgQS9TICIsInVzZXJJZCI6IjY5ODkwMSIsInRwaWQiOiIzQUI2MDJBRC03OEIxLTRFMDItOEUxNi1FMzI2NDIwNDAzRDgiLCJleHAiOjE3MTM0MjQ2MjksImlzcyI6IkVuZXJnaW5ldCIsImp0aSI6IjE1N2Q2ODdkLWQwY2UtNDk3Mi1iOTI0LTMyYTM2NDgxZGI2MSIsInRva2VuTmFtZSI6Ik1pa2tlbCIsInJvbGVzIjoiUmVhZFByaXZhdGUsIFJlYWRCdXNpbmVzcyIsImF1ZCI6IkVuZXJnaW5ldCJ9.VoP6PRoM1GFi8XPYzIjppo66fv-NrUXBq1ooJx3HdPc'
    try:
        access_token
    except NameError:
        url = "https://api.eloverblik.dk/ThirdPartyApi" + '/api/Token'
        headers = {'Authorization': 'Bearer ' + refresh_token}
        token_response = requests.get(url, headers=headers, timeout=5)
        token_response.raise_for_status()
        token_json = token_response.json()
        access_token = token_json['result']
        timelog = datetime.now() - timedelta(days=2)
    else:
        time_between_insertion = datetime.now() - timelog
        if  time_between_insertion.days > 1:
            url = "https://api.eloverblik.dk/ThirdPartyApi" + '/api/Token'
            headers = {'Authorization': 'Bearer ' + refresh_token}
            token_response = requests.get(url, headers=headers, timeout=5)
            token_response.raise_for_status()
            token_json = token_response.json()
            access_token = token_json['result']
            
            timelog = datetime.now()
    return access_token

def getMeterPoints(Customer):
    ApiKey = "3bb0bb5c-392d-4daa-8b0e-6a675387d08b"
    url = "https://superhub.dk/api/nrgiraadgivning/v2/meteringPoints"
    url = url + "?" + "apikey=" + ApiKey + "&customerKey=" + quote(Customer)
    #print(url)
    response = requests.get(url)
    df = pd.json_normalize(response.json()) 
    df['meteringPointId'] = pd.to_numeric(df['meteringPointId'])
    #print(df['meteringPointId'].to_list())
    IDs = df['meteringPointId'].to_list()
    return df, IDs

def sql_query(df):
    SERVER = "vsqlmorb"
    PORT = '1433' 
    USER = "pwu-morb"
    PASSWORD = "JGo9822YbQvlcLtx"
    DATABASE = "databox_aftagerknude"

    conn = pyodbc.connect('Driver={SQL Server};'
                        'Server=' + SERVER +';'
                        'Database='+ DATABASE +';'
                        'UID=' + USER +';'
                        'PWD=' + PASSWORD + ';')
    
    df_adr = df['Adresse']
    fejl = []
    dfs = []
    for rows in tqdm(range(df_adr.shape[0])):
        query = """
        SELECT id_lokalid, husnummer, adressebetegnelse
        FROM dar.adresse
        WHERE adressebetegnelse = '{}' 
        """.format(df_adr.iloc[rows])
        chunk = pd.read_sql(query, con=conn) 
        if chunk.empty: fejl.append(df_adr.iloc[rows])
        dfs.append(chunk)
    df = pd.concat(dfs, ignore_index=True)
    #df['ValidTo_reports'] = pd.to_datetime(df['ValidTo_reports'])
    #df = df[df['ValidTo_reports'].dt.date >= date.today()]
    df = df.reset_index()
    fejl = pd.DataFrame(fejl, columns=['Adresse'])
    return df, fejl

def BBR_api1(df):
    SERVER = "vsqlmorb"
    PORT = '1433' 
    USER = "pwu-morb"
    PASSWORD = "JGo9822YbQvlcLtx"
    DATABASE = "databox_aftagerknude"

    conn = pyodbc.connect('Driver={SQL Server};'
                        'Server=' + SERVER +';'
                        'Database='+ DATABASE +';'
                        'UID=' + USER +';'
                        'PWD=' + PASSWORD + ';')
    
    #df, _ = sql_query(df)

    df['areal'] = np.NaN
    df['anvendelseskode'] = np.NaN
    fejl = []
    dfs = []

    for rows in tqdm(range(df.shape[0])):
        query = """
        SELECT byg038samletbygningsareal, byg021bygningensanvendelse, byg007bygningsnummer, byg041bebyggetareal
        FROM bbr.bygning
        WHERE (Husnummer = '{}') AND (registreringtil IS NULL) AND (virkningtil IS NULL) AND (status = 6)
        """.format(df.iloc[rows]['husnummer'])
        chunk = pd.read_sql(query, con=conn) 
        #print(df['adressebetegnelse'].iloc[rows])
        #print(chunk['byg038samletbygningsareal'].head())
        #if chunk.empty: fejl.append(df.iloc[rows])
        #dfs.append(chunk)
        if 'byg038samletbygningsareal' in chunk.columns:
            df['areal'].iloc[rows] = chunk['byg038samletbygningsareal'].dropna().unique().sum()
        
        if pd.to_numeric(chunk['byg041bebyggetareal']).dropna().unique().sum() <= 5:
            df['areal'].iloc[rows] = chunk['byg041bebyggetareal'].dropna().unique().sum()
            
        if 'byg021bygningensanvendelse' in chunk.columns:
            df.at[rows, 'anvendelseskode'] = chunk['byg021bygningensanvendelse'].unique()
    #df = pd.concat(dfs, ignore_index=True)
    #df = df.reset_index()
    return df

def BBR_api2(df):
    SERVER = "vsqlmorb"
    PORT = '1433' 
    USER = "pwu-morb"
    PASSWORD = "JGo9822YbQvlcLtx"
    DATABASE = "databox_aftagerknude"

    conn = pyodbc.connect('Driver={SQL Server};'
                        'Server=' + SERVER +';'
                        'Database='+ DATABASE +';'
                        'UID=' + USER +';'
                        'PWD=' + PASSWORD + ';')
    
    

    for rows in tqdm(df.index[df['areal'] == 0].tolist()):
        query = """
        SELECT enh026enhedenssamledeareal, enh020enhedensanvendelse
        FROM bbr.enhed
        WHERE (id_lokalid = '{}') AND (registreringtil IS NULL) AND (virkningtil IS NULL) AND (status = 6)
        """.format(df.iloc[rows]['id_lokalid'])
        chunk = pd.read_sql(query, con=conn) 
        #print(df['adressebetegnelse'].iloc[rows])
        #print(chunk['byg038samletbygningsareal'].head())
        #if chunk.empty: fejl.append(df.iloc[rows])
        #dfs.append(chunk)
        if 'enh026enhedenssamledeareal' in chunk.columns:
            df['areal'].iloc[rows] = chunk['enh026enhedenssamledeareal'].dropna().unique().sum()
        if 'enh020enhedensanvendelse' in chunk.columns:
            df.at[rows, 'anvendelseskode'] = chunk['enh020enhedensanvendelse'].unique()
    
    for rows in tqdm(df.index[df['areal'] == 0].tolist()):
        query = """
        SELECT enh026enhedenssamledeareal, enh020enhedensanvendelse
        FROM bbr.enhed
        WHERE (id_lokalid = '{}') AND (registreringtil IS NULL) AND (virkningtil IS NULL) AND (status = 6)
        """.format(df.iloc[rows]['husnummer'])
        chunk = pd.read_sql(query, con=conn) 
        #print(df['adressebetegnelse'].iloc[rows])
        #print(chunk['byg038samletbygningsareal'].head())
        #if chunk.empty: fejl.append(df.iloc[rows])
        #dfs.append(chunk)
        if 'enh026enhedenssamledeareal' in chunk.columns:
            df['areal'].iloc[rows] = chunk['enh026enhedenssamledeareal'].dropna().unique().sum()
        if 'enh020enhedensanvendelse' in chunk.columns:
            df.at[rows, 'anvendelseskode'] = chunk['enh020enhedensanvendelse'].unique()
    #df = pd.concat(dfs, ignore_index=True)
    #df = df.reset_index()
    return df

# def BBR_api(df):

#     df, fejl = sql_query(df)

#     username = 'MikkelSchmidt2S4J8@dfdprod01.sys'
#     password = '910Mikkel14!'
#     username = 'BYCKJWQMOJ'

#     payload = ''
#     headers = { 'Accept': 'application/json',
#                 'Content-Type': 'application/json' }

#     csv = df
#     #print(csv)
#     csv['areal'] = np.NaN
#     csv['anvendeleskode'] = np.NaN
#     #print(csv)
    
#     for i in tqdm(range(len(csv))):
#         url = 'https://services.datafordeler.dk/BBR/BBRPublic/1/rest/enhed?username=' + username + '&password=' + password + '&Format=JSON&AdresseIdentificerer=' + str(csv['id_lokalid'].iloc[i])
#         response = requests.get(url, headers=headers)
#         if response.status_code != 200:
#                 response = requests.get(url, headers=headers)
#         if response.status_code == 200:
#             data = response.json()
#             #print(data)
#             df = pd.json_normalize(data,)
#             if 'enh026EnhedensSamledeAreal' in df.columns:
#                 csv['areal'].iloc[i] = df['enh026EnhedensSamledeAreal'].unique().sum()
#                 csv['anvendeleskode'].iloc[i] = df['enh020EnhedensAnvendelse'].iloc[0]

#         if csv['areal'].iloc[i] == np.NaN:
#             url = 'https://services.datafordeler.dk/BBR/BBRPublic/1/rest/bygning?username=' + username + '&password=' + password + '&Format=JSON&Husnummer=' + str(csv['husnummer'].iloc[i])
#             response = requests.get(url, headers=headers)
#             print('husnummer')
#             print(csv['adressebetegnelse'].iloc[i])
#             if response.status_code != 200:
#                     response = requests.get(url, headers=headers)
#             if response.status_code == 200:
#                 print('kode 200')
#                 data = response.json()
#                 #print(data)
#                 df = pd.json_normalize(data,)
#                 if 'byg038samletbygningsareal' in df.columns:
#                     csv['areal'].iloc[i] = df['byg038samletbygningsareal'].unique().sum()
#                     csv['anvendeleskode'].iloc[i] = df['byg021bygningensanvendelse'].iloc[0]
            
#     return csv, df

    #csv.to_csv('../Data/Master data/' + 'Furesø Kommune1' + '.csv', index=False, mode='w', header=True, encoding='UTF-8')
        #dff = df[df['id_lokalId'] == str(lokalid[0])]


def getMeterReadings(meteringPoints):
    ApiKey = "3bb0bb5c-392d-4daa-8b0e-6a675387d08b"
    From = '31122021'#(datetime.today() - timedelta(days = 1000)).strftime('%d%m%Y')
    To = datetime.today().strftime('%d%m%Y')
    df = pd.DataFrame()
    Fails = 0
    for meter in tqdm(meteringPoints):
        url = "https://superhub.dk/api/nrgiraadgivning/v2/meterreadings"
        url = url + "?" + "apikey=" + ApiKey + "&meteringpointId=" + str(meter) + "&from=" + From + "&to=" + To
        
        response = requests.get(url)
        if response.status_code != 200:
            Fails += 1
            continue
        jso = response.json()

        dff = pd.json_normalize(jso)
        
        if (dff.columns == 'meteringPoints.Production').any():
            df_meter = pd.json_normalize(jso['meteringPoints'], 'Production')
        elif (dff.columns == 'meteringPoints.Consumption').any():
            df_meter = pd.json_normalize(jso['meteringPoints'], 'Consumption') 
        else:
             print(dff.columns)
             continue
        df_meter['meter'] = jso['meteringPointId']
        df_meter['Adresse'] = jso['streetName'] + ' ' + jso['buildingNumber'] + ', ' + jso['postcode'] + ' ' + jso['cityName']
        df = pd.concat([df, df_meter], ignore_index=True)
    df['from'] = pd.to_datetime(df['from'], utc=True)
    df['meter'] = pd.to_numeric(df['meter'])
    print('Amount of fails: ' + str(Fails))
    print(url)
    return df

def get_day_moment(hour) -> str: 
    if 6 <= hour <= 18:
        return 'day'
    return 'night'

def ugeprofil(df):
            dff = df.groupby([df['from'].dt.day_name(locale='da_DK'), df['from'].dt.hour]).mean(numeric_only=True).reset_index(names=['day', 'hour'])
            dff['day_'] = dff['day']
            dff['day_'].replace({
                    "Mandag": 0,
                    "Tirsdag": 1,
                    "Onsdag": 2,
                    "Torsdag": 3,
                    "Fredag": 4,
                    "Lørdag": 5,
                    "Søndag": 6},
                    inplace=True,)
            dff.sort_values(['day_', 'hour'], ascending=True, inplace=True)
            dff['x-axis'] = dff.apply(lambda row: row['day'] + ' kl. ' + str(row['hour']), axis=1)
            return dff

def rupt(tid, df):
    nbkps = 12

    test = df['amount']
    points=np.array(test)
    n= len(points)
    if nbkps == 1:
        nbkps = np.floor(np.round(n/365*4))
    #Changepoint detection with dynamic programming search method
    model = "l1"  
    algo = rpt.Window(width=24*30, model=model, min_size=3, jump=5).fit(points)
    my_bkps = algo.predict(n_bkps=nbkps)
    return my_bkps

def besp():
    df_besp = pd.DataFrame(columns=['Adresse', 'besparelse', 'årligt forbrug', 'last', 'best', 'mean', 'bkps'])
    dfff = pd.DataFrame()
    
    for adr in tqdm(df['Adresse'].unique()):
        dff = df[df['Adresse']==adr]
        dff['day-moment'] = dff['from'].dt.hour.map(get_day_moment)
        dff = dff.groupby('from').agg({'meter': 'mean', 'amount': 'sum', 'day-moment': 'first', 'Adresse': 'first'}).reset_index()
        my_bkps = rupt('day', dff)

        value_avg_day = np.zeros(len(dff['amount']))
        #stdd_avg_day  = np.zeros(len(dff['amount']))
        vvvv = np.zeros((len(my_bkps),2),)
        k=0
        j=0
        for i in my_bkps:
            value_avg_day[j:i] = np.mean(dff['amount'][j:i])
            #stdd_avg_day[j:i]  = np.std(dff['amount'][j:i])
            vvvv[k, 0] = np.mean(dff['amount'][j:i])
            vvvv[k, 1] = i
            k += 1
            j=i
        dff['bkps'] = value_avg_day
        #print(dff['bkps'])
        
        if dff['bkps'].iloc[-1] >= dff['bkps'].max():
            df_opti = dff[dff['bkps']==dff['bkps'].iloc[-1]].groupby('from').agg({'meter': 'mean', 'amount': 'sum', 'day-moment': 'first'}).reset_index()
        else:
            df_opti = dff[dff['bkps']==dff['bkps'].min()].groupby('from').agg({'meter': 'mean', 'amount': 'sum', 'day-moment': 'first'}).reset_index()
            k=1
            while df_opti['amount'].mean() <= 0.2:
                dff1 = dff[~(dff['bkps']==dff['bkps'].min())]
                df_opti = dff1[dff1['bkps']==dff1['bkps'].min()].groupby('from').agg({'meter': 'mean', 'amount': 'sum', 'day-moment': 'first'}).reset_index()
                k += 1
                del dff1
                if (k == 10):
                    df_opti = dff[dff['bkps']==dff['bkps'].iloc[-1]].groupby('from').agg({'meter': 'mean', 'amount': 'sum', 'day-moment': 'first'}).reset_index()
                    break
                
        df_norm = dff[dff['bkps']==dff['bkps'].iloc[-1]].groupby('from').agg({'meter': 'mean', 'amount': 'sum', 'day-moment': 'first'}).reset_index()

        uge = ugeprofil(df_opti)
        uge2 = ugeprofil(df_norm) 
        #print(my_bkps)

        ugg = uge[['day', 'hour', 'amount', 'x-axis']].merge(uge2[['day', 'hour', 'amount']], how='outer', on=['day', 'hour'], suffixes=('_opti', '_now'))
        ugg['besparelse_kwh'] = ugg['amount_now'] - ugg['amount_opti']
        ttt = pd.DataFrame(data={'Adresse': [adr], 
                                 'besparelse': [ugg['besparelse_kwh'].sum()*52], 
                                 'årligt forbrug': [ugg['amount_now'].sum()*52],
                                 'last': df_norm['amount'].mean(),
                                 'best': df_opti['amount'].mean(), 
                                 'mean': dff['amount'].mean(),
                                 'meter': dff['meter'].mean(),
                                 'bkps': 0,
                                 'bkps_value': 0,
                                 'total kwh': dff['amount'].sum()} )
        ttt[['bkps', 'bkps_value']] = ttt[['bkps', 'bkps_value']].astype('object')
        ttt.at[0,'bkps'] = my_bkps
        ttt.at[0,'bkps_value'] = vvvv[:,0]
        df_besp = pd.concat([df_besp,ttt])
        dfff = pd.concat([dfff, dff])

    df_besp = df_besp[df_besp['årligt forbrug'] != 0.0]
    df_besp['%'] = df_besp.apply(lambda row: row['besparelse']/row['årligt forbrug']*100, axis=1)
    return df_besp, dfff

In [6]:
kunde = 'BallerupKommune'
df, IDs = getMeterPoints(kunde)
print(IDs)
df.head()

[571313174110568005, 571313174110568012, 571313174110568043, 571313174110568050, 571313174110568074, 571313174110568081, 571313174110568098, 571313174110568111, 571313174110568128, 571313174110568135, 571313174110568142, 571313174110568159, 571313174110568166, 571313174110568197, 571313174110577472, 571313174110585354, 571313174110585439, 571313174110585835, 571313174110598552, 571313174110598569, 571313174110598705, 571313174110598712, 571313174110598774, 571313174110598811, 571313174110598828, 571313174110598873, 571313174110599597, 571313174110600804, 571313174110600897, 571313174110600910, 571313174110606080, 571313174110606097, 571313174110606103, 571313174110607254, 571313174110609258, 571313174110612418, 571313174110614412, 571313174110614580, 571313174110616942, 571313174110616973, 571313174110617291, 571313174110619271, 571313174110619288, 571313174110620116, 571313174110620123, 571313174110627511, 571313174110627535, 571313174110628907, 571313174110630184, 571313174110630191,

,meteringPointId,streetName,buildingNumber,postcode,cityName
0,571313174110568005,Gl Rådhusvej,13,2750,Ballerup
1,571313174110568012,Gl Rådhusvej,13,2750,Ballerup
2,571313174110568043,Solvej,6,2750,Ballerup
3,571313174110568050,Torvevej,19,2740,Skovlunde
4,571313174110568074,Rugvænget,10,2750,Ballerup


In [7]:
df = getMeterReadings(IDs)

100%|██████████| 426/426 [04:59<00:00,  1.42it/s]


Amount of fails: 0
https://superhub.dk/api/nrgiraadgivning/v2/meterreadings?apikey=3bb0bb5c-392d-4daa-8b0e-6a675387d08b&meteringpointId=571313181101204514&from=31122021&to=26042023


In [8]:
df['day-moment'] = df['from'].dt.hour.map(get_day_moment)

In [9]:
df_besp, dff = besp()
dff['meter'] = pd.to_numeric(dff['meter'])
df_besp.head()

100%|██████████| 329/329 [03:18<00:00,  1.66it/s]


,Adresse,besparelse,årligt forbrug,last,best,mean,bkps,meter,bkps_value,total kwh,%
0,"Gl Rådhusvej 13, 2750 Ballerup",0.000000,260223.123333,29.389694,29.389694,37.398802,"[2445, 3720, 4720, 5285, 5955, 6855, 8510, 981...",5.713132e+17,"[46.28472801635992, 38.11173333333333, 32.3968...",429899.23,0.000000
0,"Solvej 6, 2750 Ballerup",21064.437333,53781.433333,6.093193,3.668496,5.396569,"[1045, 1870, 2520, 3245, 3695, 4550, 5255, 840...",5.713132e+17,"[5.725779904306219, 5.805369696969698, 5.39855...",61904.04,39.166746
0,"Torvevej 19, 2740 Skovlunde",17211.939333,127191.003333,14.408228,12.474740,16.417677,"[1195, 1700, 2370, 3355, 4360, 5090, 5930, 101...",5.713132e+17,"[17.83341422594142, 17.92940594059406, 17.3522...",188327.17,13.532356
0,"Rugvænget 10, 2750 Ballerup",14628.778667,200678.660000,22.708422,21.292184,27.781466,"[1275, 2200, 2745, 3765, 4205, 5185, 6045, 672...",5.713132e+17,"[30.700768627450977, 32.41547027027027, 24.740...",318681.20,7.289653
0,"Lundebjerg 72, 2740 Skovlunde",74677.458556,278953.357222,31.939980,23.240565,30.436013,"[1305, 2205, 3180, 3745, 4370, 5290, 5885, 727...",5.713132e+17,"[30.99321072796935, 33.47655555555556, 29.7150...",349131.50,26.770590


In [10]:
df_sql, _ = sql_query(df_besp)

  0%|          | 0/323 [00:00<?, ?it/s]C:\Users\misc\AppData\Local\Temp\ipykernel_23496\3170966637.py:60: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chunk = pd.read_sql(query, con=conn)
  0%|          | 1/323 [00:09<48:57,  9.12s/it]C:\Users\misc\AppData\Local\Temp\ipykernel_23496\3170966637.py:60: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chunk = pd.read_sql(query, con=conn)
  1%|          | 2/323 [00:09<21:02,  3.93s/it]C:\Users\misc\AppData\Local\Temp\ipykernel_23496\3170966637.py:60: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider usi

In [12]:
df_sql.head()

,index,id_lokalid,husnummer,adressebetegnelse
0,0,0a3f50a2-facc-32b8-e044-0003ba298018,0a3f507b-67d8-32b8-e044-0003ba298018,"Gl Rådhusvej 13, 2750 Ballerup"
1,1,2d5f65ce-fb97-4d79-ad1f-7e53cac22667,0a3f507b-7635-32b8-e044-0003ba298018,"Solvej 6, 2750 Ballerup"
2,2,0a3f50a3-3408-32b8-e044-0003ba298018,0a3f507b-88f9-32b8-e044-0003ba298018,"Torvevej 19, 2740 Skovlunde"
3,3,0a3f50a3-1359-32b8-e044-0003ba298018,0a3f507b-739d-32b8-e044-0003ba298018,"Rugvænget 10, 2750 Ballerup"
4,4,2e350495-4b1f-45b2-9bd0-acf015bd039e,0a3f507b-8412-32b8-e044-0003ba298018,"Lundebjerg 72, 2740 Skovlunde"


In [65]:
csv = BBR_api1(df_sql)


  0%|          | 0/409 [00:00<?, ?it/s]C:\Users\misc\AppData\Local\Temp\ipykernel_23496\3892841152.py:96: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chunk = pd.read_sql(query, con=conn)
  0%|          | 1/409 [00:03<24:07,  3.55s/it]C:\Users\misc\AppData\Local\Temp\ipykernel_23496\3892841152.py:96: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chunk = pd.read_sql(query, con=conn)
  0%|          | 2/409 [00:06<22:05,  3.26s/it]C:\Users\misc\AppData\Local\Temp\ipykernel_23496\3892841152.py:96: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider usi

DatabaseError: Execution failed on sql: 
        SELECT byg038samletbygningsareal, byg021bygningensanvendelse, byg007bygningsnummer, byg041bebyggetareal
        FROM bbr.bygning
        WHERE (Husnummer = '0a3f507b-6608-32b8-e044-0003ba298018') AND (registreringtil IS NULL) AND (virkningtil IS NULL) AND (status = 6)
        
('01000', '[01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionRead (recv()). (10053) (SQLExecDirectW); [01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]General network error. Check your network documentation. (11)')
unable to rollback

In [ ]:
csv.drop_duplicates(subset=['adressebetegnelse','areal']).sort_values('areal').head(60)


,index,id_lokalid,husnummer,adressebetegnelse,areal,anvendelseskode
102,102,0a3f50a3-023f-32b8-e044-0003ba298018,0a3f507b-6b3f-32b8-e044-0003ba298018,"Hold-An Vej 22, 2750 Ballerup",0.0,[]
132,132,01427206-70e4-49fb-a852-efdde150364d,48f90100-615a-4b2b-a312-757cf0aab280,"Marbækvej 2, 2750 Ballerup",0.0,[]
127,127,703f3355-d651-4969-b2dc-730ddb3534a0,5b74086e-d3df-4856-8337-989ff9f46206,"Sydbuen 1, 2750 Ballerup",0.0,[930]
268,268,9a4474d4-68bd-4e99-8ed6-b2489e604031,0a3f507b-8c64-32b8-e044-0003ba298018,"Klakkebjerg 8, 2750 Ballerup",0.0,[]
122,122,7d162ba9-b12e-47e3-8ce2-944b64dff5fb,69e9fa68-1e74-42f1-9c0c-71d64890017b,"Bybjergvej 8, 2740 Skovlunde",0.0,[]
376,376,4a08e7a8-4bc9-4794-8c45-5f3cc84eba8e,0a3f507b-747d-32b8-e044-0003ba298018,"Sct. Jacobsvej 5, 2750 Ballerup",0.0,[]
281,281,0a3f50a3-2d40-32b8-e044-0003ba298018,0a3f507b-850a-32b8-e044-0003ba298018,"Lystoften 24, 2740 Skovlunde",0.0,[]
285,285,0d54d21a-1468-49dd-8ebe-c21785d4836c,b7a4112a-28e8-49db-898b-d912c92e70ae,"Lundebjerggårdsvej 5, 2740 Skovlunde",0.0,[]
291,291,f7af2309-5ddd-42bd-9572-85107e607a74,0a3f507b-8c17-32b8-e044-0003ba298018,"Jungshøjvej 1, 2760 Måløv",0.0,[]
105,105,3b05b803-2516-4b1f-abde-ce64d7acd895,482260ce-305b-4be8-b164-d49b7dcf22c8,"Jonstrupvej 71A, 2750 Ballerup",0.0,[]


In [45]:
csvv = BBR_api2(csv.drop_duplicates(subset=['adressebetegnelse','areal']))

  0%|          | 0/47 [00:00<?, ?it/s]C:\Users\misc\AppData\Local\Temp\ipykernel_23496\1609767777.py:130: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chunk = pd.read_sql(query, con=conn)
  2%|▏         | 1/47 [00:02<01:34,  2.06s/it]C:\Users\misc\AppData\Local\Temp\ipykernel_23496\1609767777.py:130: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chunk = pd.read_sql(query, con=conn)
  4%|▍         | 2/47 [00:03<01:29,  1.99s/it]C:\Users\misc\AppData\Local\Temp\ipykernel_23496\1609767777.py:130: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider usi

IndexError: single positional indexer is out-of-bounds

In [44]:
csvv.drop_duplicates(subset=['adressebetegnelse','areal']).sort_values('areal').head(50)

,index,id_lokalid,husnummer,adressebetegnelse,areal,anvendelseskode
102,102,0a3f50a3-023f-32b8-e044-0003ba298018,0a3f507b-6b3f-32b8-e044-0003ba298018,"Hold-An Vej 22, 2750 Ballerup",0.0,[]
132,132,01427206-70e4-49fb-a852-efdde150364d,48f90100-615a-4b2b-a312-757cf0aab280,"Marbækvej 2, 2750 Ballerup",0.0,[]
127,127,703f3355-d651-4969-b2dc-730ddb3534a0,5b74086e-d3df-4856-8337-989ff9f46206,"Sydbuen 1, 2750 Ballerup",0.0,[]
268,268,9a4474d4-68bd-4e99-8ed6-b2489e604031,0a3f507b-8c64-32b8-e044-0003ba298018,"Klakkebjerg 8, 2750 Ballerup",0.0,[]
122,122,7d162ba9-b12e-47e3-8ce2-944b64dff5fb,69e9fa68-1e74-42f1-9c0c-71d64890017b,"Bybjergvej 8, 2740 Skovlunde",0.0,[]
376,376,4a08e7a8-4bc9-4794-8c45-5f3cc84eba8e,0a3f507b-747d-32b8-e044-0003ba298018,"Sct. Jacobsvej 5, 2750 Ballerup",0.0,[]
281,281,0a3f50a3-2d40-32b8-e044-0003ba298018,0a3f507b-850a-32b8-e044-0003ba298018,"Lystoften 24, 2740 Skovlunde",0.0,[]
285,285,0d54d21a-1468-49dd-8ebe-c21785d4836c,b7a4112a-28e8-49db-898b-d912c92e70ae,"Lundebjerggårdsvej 5, 2740 Skovlunde",0.0,[]
291,291,f7af2309-5ddd-42bd-9572-85107e607a74,0a3f507b-8c17-32b8-e044-0003ba298018,"Jungshøjvej 1, 2760 Måløv",0.0,[]
105,105,3b05b803-2516-4b1f-abde-ce64d7acd895,482260ce-305b-4be8-b164-d49b7dcf22c8,"Jonstrupvej 71A, 2750 Ballerup",0.0,[]


In [220]:
csv = csv.drop_duplicates(subset=['adressebetegnelse','areal'])
df_besp1 = df_besp.merge(csv[['adressebetegnelse', 'areal', 'anvendelseskode']], how='outer', left_on='Adresse', right_on='adressebetegnelse')

df_besp1.head(20)

,Adresse,besparelse,årligt forbrug,last,best,mean,bkps,meter,bkps_value,total kwh,%,adressebetegnelse,areal,anvendelseskode
0,"Gl Rådhusvej 13, 2750 Ballerup",0.000000,260096.546667,29.714466,29.714466,37.479331,"[2445, 3720, 4720, 5285, 5955, 6855, 8510, 981...",5.713132e+17,"[46.28472801635992, 38.11173333333333, 32.3968...",427226.89,0.000000,"Gl Rådhusvej 13, 2750 Ballerup",2239.0,"[532, 930, 420]"
1,"Solvej 6, 2750 Ballerup",18487.404000,51204.400000,6.040184,3.668496,5.388919,"[1045, 1870, 2520, 3245, 3695, 4550, 5255, 840...",5.713132e+17,"[5.725779904306219, 5.805369696969698, 5.39855...",61298.95,36.105108,"Solvej 6, 2750 Ballerup",562.0,"[930, 420]"
2,"Torvevej 19, 2740 Skovlunde",15016.542667,124995.606667,14.355205,12.474740,16.432585,"[1195, 1700, 2370, 3355, 4360, 5090, 5930, 101...",5.713132e+17,"[17.83341422594142, 17.92940594059406, 17.3522...",186920.65,12.013656,"Torvevej 19, 2740 Skovlunde",4794.0,"[321, 590, 930]"
3,"Rugvænget 10, 2750 Ballerup",9139.312000,195189.193333,22.527419,21.292184,27.817438,"[1275, 2200, 2745, 3765, 4205, 5185, 6045, 672...",5.713132e+17,"[30.700768627450977, 32.41547027027027, 24.740...",316423.36,4.682284,"Rugvænget 10, 2750 Ballerup",8431.0,"[421, 930, 439, 441, 950, 590]"
4,"Lundebjerg 72, 2740 Skovlunde",73381.221333,277657.120000,32.033569,23.240565,30.434962,"[1305, 2205, 3180, 3745, 4370, 5290, 5885, 727...",5.713132e+17,"[30.99321072796935, 33.47655555555556, 29.7150...",346197.69,26.428719,"Lundebjerg 72, 2740 Skovlunde",9923.0,"[421, 533, 429]"
5,"Kratvej 14, 2760 Måløv",68844.753714,265327.065714,30.698409,22.471494,30.199561,"[865, 1330, 2290, 3840, 4445, 5255, 6060, 7035...",5.713132e+17,"[34.435653179190744, 30.66698924731183, 33.145...",343520.01,25.947128,"Kratvej 14, 2760 Måløv",6151.0,"[421, 930]"
6,"Ejbyvej 47, 2740 Skovlunde",100752.513333,276227.943333,32.182796,20.075193,40.077519,"[860, 2370, 3355, 4360, 5165, 5830, 6880, 7610...",5.713132e+17,"[45.21204651162791, 43.176741721854306, 38.106...",455881.78,36.474410,"Ejbyvej 47, 2740 Skovlunde",12134.0,"[930, 421, 533]"
7,"Magleparken 10, 2750 Ballerup",49426.017333,217923.853333,24.481478,19.119017,34.785089,"[1370, 2200, 2740, 3285, 4190, 5095, 5930, 688...",5.713132e+17,"[42.122737226277366, 38.178590361445785, 30.53...",395680.39,22.680407,"Magleparken 10, 2750 Ballerup",3039.0,[590]
8,"Bybjergvej 25, 2740 Skovlunde",0.000000,146493.186667,16.737363,16.737363,19.135485,"[870, 4065, 5775, 7130, 7560, 8500, 9745, 1092...",5.713132e+17,"[19.878999999999998, 18.48045070422535, 19.419...",217666.14,0.000000,"Bybjergvej 25, 2740 Skovlunde",5821.0,"[421, 910, 930, 940, 160, 950]"
9,"Platanbuen 1, 2750 Ballerup",70586.833571,325902.023571,37.683012,28.226602,38.211105,"[370, 865, 2205, 2595, 3740, 5260, 5880, 6800,...",5.713132e+17,"[37.02762162162162, 44.49725252525253, 40.1164...",434651.32,21.658912,"Platanbuen 1, 2750 Ballerup",14733.0,"[930, 421, 533]"


In [232]:
df_besp1['areal'].sort_values().value_counts().head(50)

0.0       47
119.0      3
102.0      3
80.0       3
140.0      3
158.0      3
180.0      3
126.0      3
127.0      3
132.0      2
135.0      2
124.0      2
94.0       2
148.0      2
151.0      2
783.0      2
129.0      2
96.0       2
138.0      2
128.0      2
88.0       2
85.0       2
84.0       2
517.0      2
89.0       2
70.0       2
1768.0     1
2093.0     1
2688.0     1
2239.0     1
1466.0     1
2201.0     1
2190.0     1
1467.0     1
1646.0     1
1590.0     1
2001.0     1
1649.0     1
1654.0     1
1826.0     1
1804.0     1
1665.0     1
1713.0     1
1731.0     1
1084.0     1
1260.0     1
773.0      1
600.0      1
631.0      1
635.0      1
Name: areal, dtype: int64

In [222]:
df_besp1.to_csv('../Data/besp/'+kunde+'.csv', header=True, sep=',', index=False)
dff.to_csv('../Data/timeforbrug/'+kunde+'.csv', header=True, sep=',', index=False) 

In [162]:
df_besp = df_besp1
df_besp.shape
df_bbr.shape

(1, 31)

In [161]:
#csv, df_bbr = BBR_api(df_besp)

  0%|          | 0/323 [00:00<?, ?it/s]C:\Users\misc\AppData\Local\Temp\ipykernel_26464\3776361956.py:60: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chunk = pd.read_sql(query, con=conn)
  0%|          | 1/323 [00:00<01:22,  3.90it/s]C:\Users\misc\AppData\Local\Temp\ipykernel_26464\3776361956.py:60: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chunk = pd.read_sql(query, con=conn)
  1%|          | 2/323 [00:00<01:20,  4.00it/s]C:\Users\misc\AppData\Local\Temp\ipykernel_26464\3776361956.py:60: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider usi

,Adresse,besparelse,årligt forbrug,last,best,mean,bkps,meter,bkps_value,total kwh,%,adressebetegnelse,areal,anvendeleskode
0,"Gl Rådhusvej 13, 2750 Ballerup",0.000000,259497.680000,29.492138,29.492138,37.487211,"[2445, 3720, 4720, 5285, 5955, 6855, 8510, 981...",5.713132e+17,"[46.28472801635992, 38.11173333333333, 32.3968...",426417.03,0.000000,"Gl Rådhusvej 13, 2750 Ballerup",5975.0,532
1,"Solvej 6, 2750 Ballerup",18492.430667,51209.426667,5.996885,3.668496,5.388264,"[1045, 1870, 2520, 3245, 3695, 4550, 5255, 840...",5.713132e+17,"[5.725779904306219, 5.805369696969698, 5.39855...",61162.18,36.111380,"Solvej 6, 2750 Ballerup",NaN,NaN
2,"Torvevej 19, 2740 Skovlunde",14179.082667,124158.146667,14.161923,12.474740,16.429893,"[1195, 1700, 2370, 3355, 4360, 5090, 5930, 101...",5.713132e+17,"[17.83341422594142, 17.92940594059406, 17.3522...",186495.72,11.420179,"Torvevej 19, 2740 Skovlunde",6242.0,360
3,"Rugvænget 10, 2750 Ballerup",7584.772000,193634.653333,22.078744,21.292184,27.812575,"[1275, 2200, 2745, 3765, 4205, 5185, 6045, 672...",5.713132e+17,"[30.700768627450977, 32.41547027027027, 24.740...",315700.54,3.917053,"Rugvænget 10, 2750 Ballerup",7439.0,350
4,"Lundebjerg 72, 2740 Skovlunde",73013.465778,277289.364444,31.918706,23.240565,30.417506,"[1305, 2205, 3180, 3745, 4370, 5290, 5885, 727...",5.713132e+17,"[30.99321072796935, 33.47655555555556, 29.7150...",345269.11,26.331145,"Lundebjerg 72, 2740 Skovlunde",8759.0,421
5,"Kratvej 14, 2760 Måløv",68501.380381,264983.692381,30.543035,22.471494,30.184120,"[865, 1330, 2290, 3840, 4445, 5255, 6060, 7035...",5.713132e+17,"[34.435653179190744, 30.66698924731183, 33.145...",342619.95,25.851168,"Kratvej 14, 2760 Måløv",8934.0,421
6,"Ejbyvej 47, 2740 Skovlunde",99065.460000,274540.890000,31.649651,20.075193,40.069975,"[860, 2370, 3355, 4360, 5165, 5830, 6880, 7610...",5.713132e+17,"[45.21204651162791, 43.176741721854306, 38.106...",454834.29,36.084046,"Ejbyvej 47, 2740 Skovlunde",9756.0,421
7,"Magleparken 10, 2750 Ballerup",45695.537333,214193.373333,23.860665,19.119017,34.783029,"[1370, 2200, 2740, 3285, 4190, 5095, 5930, 688...",5.713132e+17,"[42.122737226277366, 38.178590361445785, 30.53...",394822.16,21.333777,"Magleparken 10, 2750 Ballerup",3676.0,590
8,"Bybjergvej 25, 2740 Skovlunde",0.000000,147015.266667,16.671578,16.671578,19.138057,"[870, 4065, 5775, 7130, 7560, 8500, 9745, 1092...",5.713132e+17,"[19.878999999999998, 18.48045070422535, 19.419...",217236.09,0.000000,"Bybjergvej 25, 2740 Skovlunde",3108.0,442
9,"Platanbuen 1, 2750 Ballerup",70168.985714,325484.175714,37.504087,28.226602,38.192975,"[370, 865, 2205, 2595, 3740, 5260, 5880, 6800,...",5.713132e+17,"[37.02762162162162, 44.49725252525253, 40.1164...",433528.46,21.558340,"Platanbuen 1, 2750 Ballerup",14733.0,421


In [ ]:
#df_besp.head()

In [ ]:
dfff = dff[dff['Adresse']=='Vandmanden 5, 9200 Aalborg SV'].groupby('from').agg({'meter': 'mean', 'amount': 'sum', 'bkps': 'sum'}).reset_index()

fig, ax = plt.subplots(figsize=(14,4)) 
ax.plot(dfff['from'], dfff['amount'], linewidth=0.3)
ax.plot(dfff['from'], dfff['bkps'])
ax.plot(dfff['from'][dfff['bkps']==dfff['bkps'].min()], dfff['bkps'][dfff['bkps']==dfff['bkps'].min()], linewidth=6)


In [ ]:
dfff.head()

In [72]:
def sql_query(df):
    df_adr = df['Adresse']
    fejl = []
    dfs = []
    for rows in tqdm(range(df_adr.shape[0])):
        query = """
        SELECT id_lokalid, husnummer
        FROM dar.adresse
        WHERE adressebetegnelse = '{}' 
        """.format(df_adr.iloc[rows])
        chunk = pd.read_sql(query, con=conn) 
        if chunk.empty: fejl.append(df_adr.iloc[rows])
        dfs.append(chunk)
    df = pd.concat(dfs, ignore_index=True)
    #df['ValidTo_reports'] = pd.to_datetime(df['ValidTo_reports'])
    #df = df[df['ValidTo_reports'].dt.date >= date.today()]
    df = df.reset_index()
    fejl = pd.DataFrame(fejl, columns=['Adresse'])
    return df, fejl

In [73]:
SERVER = "vsqlmorb"
PORT = '1433' 
USER = "pwu-morb"
PASSWORD = "JGo9822YbQvlcLtx"
DATABASE = "databox_aftagerknude"

conn = pyodbc.connect('Driver={SQL Server};'
                    'Server=' + SERVER +';'
                    'Database='+ DATABASE +';'
                    'UID=' + USER +';'
                    'PWD=' + PASSWORD + ';')
cursor = conn.cursor()

df, fejl = sql_query(df_besp)

  0%|          | 0/323 [00:00<?, ?it/s]C:\Users\misc\AppData\Local\Temp\ipykernel_26464\1323498115.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chunk = pd.read_sql(query, con=conn)
  0%|          | 1/323 [00:00<01:26,  3.70it/s]C:\Users\misc\AppData\Local\Temp\ipykernel_26464\1323498115.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chunk = pd.read_sql(query, con=conn)
  1%|          | 2/323 [00:00<01:21,  3.96it/s]C:\Users\misc\AppData\Local\Temp\ipykernel_26464\1323498115.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider usi

In [78]:
df.head()

,index,id_lokalid,husnummer
0,0,0a3f50a2-facc-32b8-e044-0003ba298018,0a3f507b-67d8-32b8-e044-0003ba298018
1,1,2d5f65ce-fb97-4d79-ad1f-7e53cac22667,0a3f507b-7635-32b8-e044-0003ba298018
2,2,0a3f50a3-3408-32b8-e044-0003ba298018,0a3f507b-88f9-32b8-e044-0003ba298018
3,3,0a3f50a3-1359-32b8-e044-0003ba298018,0a3f507b-739d-32b8-e044-0003ba298018
4,4,2e350495-4b1f-45b2-9bd0-acf015bd039e,0a3f507b-8412-32b8-e044-0003ba298018


In [76]:
fejl.shape

(25, 1)

In [104]:
csv, df = BBR_api(df_besp)

  0%|          | 0/323 [00:00<?, ?it/s]C:\Users\misc\AppData\Local\Temp\ipykernel_26464\1450996751.py:60: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chunk = pd.read_sql(query, con=conn)
  0%|          | 1/323 [00:00<01:28,  3.65it/s]C:\Users\misc\AppData\Local\Temp\ipykernel_26464\1450996751.py:60: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chunk = pd.read_sql(query, con=conn)
  1%|          | 2/323 [00:00<01:46,  3.02it/s]C:\Users\misc\AppData\Local\Temp\ipykernel_26464\1450996751.py:60: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider usi

In [106]:
csv.head(50)

,index,id_lokalid,husnummer,adressebetegnelse,areal,anvendeleskode
0,0,0a3f50a2-facc-32b8-e044-0003ba298018,0a3f507b-67d8-32b8-e044-0003ba298018,"Gl Rådhusvej 13, 2750 Ballerup",5975.0,532
1,1,2d5f65ce-fb97-4d79-ad1f-7e53cac22667,0a3f507b-7635-32b8-e044-0003ba298018,"Solvej 6, 2750 Ballerup",NaN,NaN
2,2,0a3f50a3-3408-32b8-e044-0003ba298018,0a3f507b-88f9-32b8-e044-0003ba298018,"Torvevej 19, 2740 Skovlunde",6242.0,360
3,3,0a3f50a3-1359-32b8-e044-0003ba298018,0a3f507b-739d-32b8-e044-0003ba298018,"Rugvænget 10, 2750 Ballerup",7439.0,350
4,4,2e350495-4b1f-45b2-9bd0-acf015bd039e,0a3f507b-8412-32b8-e044-0003ba298018,"Lundebjerg 72, 2740 Skovlunde",8759.0,421
5,5,ee7ef6dc-0eba-4f74-bfe5-056f200c7030,0a3f507b-8dff-32b8-e044-0003ba298018,"Kratvej 14, 2760 Måløv",8934.0,421
6,6,e7d53961-ddd0-42ea-a302-9e425dc83f86,0a3f507b-7f7f-32b8-e044-0003ba298018,"Ejbyvej 47, 2740 Skovlunde",9756.0,421
7,7,0a3f50a3-0b6e-32b8-e044-0003ba298018,0a3f507b-707c-32b8-e044-0003ba298018,"Magleparken 10, 2750 Ballerup",3676.0,590
8,8,fda1725c-1f12-4511-bba8-e8f32e0d7a9b,0a3f507b-7e73-32b8-e044-0003ba298018,"Bybjergvej 25, 2740 Skovlunde",3108.0,442
9,9,fda1725c-1f12-4511-bba8-e8f32e0d7a9b,0a3f507b-7e73-32b8-e044-0003ba298018,"Bybjergvej 25, 2740 Skovlunde",3108.0,442


In [83]:
df.head()

""


In [55]:
data = pd.read_csv('../MeteringPoints.csv', sep=';')
data2 = pd.read_csv('../Meterdata.csv', sep=';')
data = data.merge(data2)
data = data[['MålepunktsID', 'Juridisk_kontakt_By', 'Fra_dato', 'Til_dato', 'Mængde']]
data.columns = ['meter', 'Adresse', 'from', 'to', 'amount']
data['from'] = pd.to_datetime(data['from'])
#data['amount'] = pd.to_numeric(data['amount'])
data['day-moment'] = data['from'].dt.hour.map(get_day_moment)

data.head()

,meter,Adresse,from,to,amount,day-moment
0,571313123200330480,Mårslet,2019-04-19 00:00:00,19-04-2019 01:00,0.08,night
1,571313123200330480,Mårslet,2019-04-19 01:00:00,19-04-2019 02:00,0.07,night
2,571313123200330480,Mårslet,2019-04-19 02:00:00,19-04-2019 03:00,0.06,night
3,571313123200330480,Mårslet,2019-04-19 03:00:00,19-04-2019 04:00,0.07,night
4,571313123200330480,Mårslet,2019-04-19 04:00:00,19-04-2019 05:00,0.05,night


In [56]:
df = data
df_besp, dff = besp()
dff['meter'] = pd.to_numeric(dff['meter'])
kunde = 'Privat_elforbrug_test'
df_besp.head()
dff = dff.iloc[889:]
dff.head()

  0%|          | 0/1 [00:00<?, ?it/s]C:\Users\misc\AppData\Local\Temp\ipykernel_26464\2830085382.py:80: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dff = df.groupby([df['from'].dt.day_name(locale='da_DK'), df['from'].dt.hour]).mean().reset_index(names=['day', 'hour'])
C:\Users\misc\AppData\Local\Temp\ipykernel_26464\2830085382.py:80: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dff = df.groupby([df['from'].dt.day_name(locale='da_DK'), df['from'].dt.hour]).mean().reset_index(names=['day', 'hour'])
C:\Users\misc\AppData\Local\Temp\ipykernel_26464\2830085382.py:169: FutureWarning: The frame.append method is

,from,meter,amount,day-moment,Adresse,bkps
889,2019-04-24 00:00:00,5.713131e+17,0.08,night,Mårslet,0.96978
890,2019-04-24 01:00:00,5.713131e+17,0.07,night,Mårslet,0.96978
891,2019-04-24 02:00:00,5.713131e+17,0.07,night,Mårslet,0.96978
892,2019-04-24 03:00:00,5.713131e+17,0.07,night,Mårslet,0.96978
893,2019-04-24 04:00:00,5.713131e+17,0.10,night,Mårslet,0.96978
